In [ ]:
import pandas as pd
import bibtexparser
import ast

info = ['title', 'author', 'year', 'publisher', 'doi']

In [ ]:
with open(f"../Data/Mendeley Notes.bib") as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)

df = pd.DataFrame(bib_database.entries)

df = df.replace(r'\n',' ', regex=True) 
df['note'] = df['note'].str.replace('"',"'") 
df['note'] = df['note'].str.replace('\\',"") 
df['note'] = df['note'].str.replace('<br/>','')
df['note'] = df['note'].str.replace('obs','Obs')
df['note'] = df['note'].str.replace('comment','Obs')

In [ ]:
list_dict = []
i = 1

for index,row in df.iterrows():
    row_dict = ast.literal_eval(row['note'])

    for iinfo in info:
        row_dict[iinfo] = row[iinfo]

    if row_dict['id']!=i:
        i = row_dict['id']
    
    list_dict.append(row_dict)
    i += 1

notes_df = pd.DataFrame(list_dict)

In [ ]:
papers_df = pd.read_csv('../Data/Papers - After Abstract Filter.csv')

In [ ]:
papers_df['Title'] = papers_df['Title'].str.lower()
notes_df['title'] = notes_df['title'].str.lower()

In [ ]:
full_df_title = papers_df.join(notes_df.set_index('title'), on='Title')
full_df_title['id'].count()

In [ ]:
full_df_title

In [ ]:
full_df_title[full_df_title['id'].isna()]